---
title: APIs
subtitle: "Application Programming Interfaces"
author:
  - name: Charles Pletcher
    affiliations: Tufts University
    orcid: 0000-0003-2734-5511
    email: charles.pletcher@tufts.edu
license:
  code: MIT
date: 2025-03-31
---

# Application Programming Interfaces

You will hear the term "API" in many different contexts when it comes to computer programming. Already in this class, we have discussed Python's API for the file system.

Often, however, when someone mentions an API, they are referring to a web-based API that is usually accessed over HTTP(S). You might have heard about the kerfuffle when Twitter shut down much of the access to its API, or when Reddit did the same thing a few years earlier. These APIs are servers that provide _interfaces_ (the "I" in "API") to a platform's data.

As you probably noticed while reading [Walker 2019](https://studentwork.prattsi.org/dh/2019/05/13/getting-data-for-digital-humanities-with-apis/), it is not exactly uncommon for references to APIs to become out of date.

Luckily, we can still use the API provided by the [Digital Public Library of America](https://dp.la) for our work for this class.

We'll be working with the Python [Requests](https://docs.python-requests.org/en/latest/) library, which provides its own easy-to-use API for making HTTP requests. In other words, it's APIs all the way down.

## Getting an access token

In [ ]:
# Getting an access token

## Making your first request

In [ ]:
# Making your first request

## Reading responses

In [ ]:
# Reading responses

## Constructing queries

In [ ]:
# Constructing queries

## Your turn

## Readings

- [Walker 2019](https://studentwork.prattsi.org/dh/2019/05/13/getting-data-for-digital-humanities-with-apis/): Getting Data for Digital Humanities with APIs: A Gentle Introduction